## Exercise: Horovod with Petastorm for training a deep learning model

In this exercise we are going to build a model on the Boston housing dataset and distribute the deep learning training process using both HorovodRunner and Petastorm.

**Required Libraries**: 
* `petastorm==0.8.2` via PyPI

Run the following cell to set up our environment.

In [0]:
%run "./Includes/Classroom-Setup"

## 1. Load and process data

We again load the Boston housing data. However, as we saw in the demo, for Horovod we want to shard the data before passing into HorovodRunner. 

For the `get_dataset` function below, load the data, split into 80/20 train-test, standardize the features and return train and test sets.

In [0]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def get_dataset(rank=0, size=1):
  scaler = StandardScaler()
  
  boston_housing = load_boston()

  # split 80/20 train-test
  X_train, X_test, y_train, y_test = train_test_split(boston_housing.data,
                                                          boston_housing.target,
                                                          test_size=0.2,
                                                          random_state=1)
  
  scaler.fit(X_train)
  X_train = scaler.transform(X_train[rank::size])
  y_train = y_train[rank::size]
  X_test = scaler.transform(X_test[rank::size])
  y_test = y_test[rank::size]
  
  return (X_train, y_train), (X_test, y_test)

##2. Build Model

Using the same model from earlier, let's define our model architecture

In [0]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
# tf.set_random_seed(42) # For reproducibility
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model():
  return Sequential([Dense(50, input_dim=13, activation='relu'),
                    Dense(20, activation='relu'),
                    Dense(1, activation='linear')])

## 3. Horovod

In order to distribute the training of our Keras model with Horovod, we must define our `run_training_horovod` training function

In [0]:
# TODO
import horovod.tensorflow.keras as hvd
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import *

ml_working_path = "dbfs:/pinky.gtm@mail.kmutt.ac.th"

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  print(f"Rank is: {hvd.rank()}")
  print(f"Size is: {hvd.size()}")
  
  (X_train, y_train), (X_test, y_test) = get_dataset(hvd.rank(), hvd.size())
  
  model = build_model()
  from tensorflow.keras import optimizers
  optimizer = optimizers.Adam(lr=0.001*hvd.size())
  optimizer = hvd.DistributedOptimizer(optimizer)
  
  model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])
  checkpoint_dir = f"{ml_working_path}/horovod_checkpoint_weights_lab.ckpt"
  
  callbacks = [
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
    hvd.callbacks.MetricAverageCallback(),
    hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=10, verbose=1)
  ]
  
  # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
  if hvd.rank() == 0:
    callbacks.append(ModelCheckpoint(checkpoint_dir, save_weights_only=True))
  
  # (make sure you use batch_size of 16 for the learning rate warmup callback, or else you might get a division by 0 error with this small dataset)
  history = model.fit(X_train, y_train, batch_size=16, callbacks=callbacks)

Let's now run our model on all workers.

In [0]:
# TODO
from sparkdl import HorovodRunner

hr = HorovodRunner(np=-1)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'ModelCheckpoint', 'hvd', 'build_model', 'print', 'tf', 'get_dataset', 'ml_working_path'}.
The pickled object size is 3377 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_e9080ae27b81/id_rsa', 'bash', '/tmp/HorovodRunner_e9080ae27b81/launch.sh'].

[1,0]<stdout>:Rank is: 0
[1,0]<stdout>:Size is: 1
[1,0]<stderr>:2022-03-09 10:46:17.683058: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 FMA
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-03-09 10:46:17.689324: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2397220000 Hz
[1,0]<stderr>:2022-03-09 10:46:17.689734: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558a2ad04aa0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2022-03-09 10:46:17.689777: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stdout>:[1,0]<stdout>: 1/26 [>.............................] - ETA: 0s - loss: 476.8183 - mse: 476.8183[1,0]<stdout>:[1,0]<stdout>:26/26 [==============================] - 0s 2ms/step - loss: 532.5295 - mse: 532.5295

## 4. Horovod with Petastorm

We're now going to build a distributed deep learning model capable of handling data in Apache Parquet format. To do so, we can use Horovod along with Petastorm. 

First let's load the Boston housing data, and create a Spark DataFrame from the training data.

In [0]:
import pandas as pd

boston_housing = load_boston()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(boston_housing.data,
                                                        boston_housing.target,
                                                        test_size=0.2,
                                                        random_state=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# concatenate our features and label, then create a Spark DataFrame from our Pandas DataFrame.
data = pd.concat([pd.DataFrame(X_train, columns=boston_housing.feature_names), 
                  pd.DataFrame(y_train, columns=["label"])], axis=1)
trainDF = spark.createDataFrame(data)
display(trainDF)

CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,label
-0.3892493956586173,-0.49559343450001486,-0.6092897798640649,-0.2932942300427058,-0.8995829271341742,-0.14496758382355698,-2.1500295879693434,0.8944552840373514,-0.7463298414786979,-1.0085076489790397,-0.24857777096080874,0.2867418155119902,-0.9668501639955871,25.3
-0.38783183560964163,0.5792387940699122,-0.8695263265126643,-0.2932942300427058,-0.8567563464869284,-0.17983229492468977,-1.357820264574482,1.8829028457066803,-0.16959446283027801,-0.7064128071616964,0.582147206257553,0.36669519266481015,-0.8211678934064371,23.3
1.4355499313251183,-0.49559343450001486,1.0266916566515687,-0.2932942300427058,1.2588767374870105,-1.4407726797489904,1.0573665664833036,-1.1329503169261308,1.6759587488446654,1.5563367923329132,0.8129041443737646,0.4347266572189193,2.5017753262222664,7.2
-0.3985582224203851,-0.49559343450001486,0.2562160382031871,-0.2932942300427058,-0.9938014045581149,-0.05344771718308261,-0.4990092723986364,0.5608028953903854,-0.5156356900193299,-0.031141984275870463,0.12063333002513002,0.3198825450295062,-0.06084518595068373,21.2
0.5576829063390698,-0.49559343450001486,1.0266916566515687,-0.2932942300427058,0.26530006647090937,-1.022396146535397,0.09339504465327303,-0.8320589296545178,1.6759587488446654,1.5563367923329132,0.8129041443737646,-3.8664587826369567,0.6079058085633184,11.7
-0.2653404170456044,-0.49559343450001486,1.2430681111683812,3.409545424246455,0.4451717051893416,-0.02729918385723334,0.8645722621172975,-0.9572021784240574,-0.5156356900193299,-0.0015248429212289733,-1.7254221749045622,-0.19943159275652514,-1.0098611200742884,27.0
-0.3981192713936366,-0.49559343450001486,-1.2598811464855633,-0.2932942300427058,-0.5569702819562086,-0.16821072455764552,0.030298726860762033,-0.2579389026703073,-0.7463298414786979,-1.2454447798161714,-0.2947291585840501,0.3276481945204098,0.051538279932374795,29.6
-0.39222462980001954,-0.49559343450001486,-0.3680592731392395,-0.2932942300427058,-0.28288016581383557,0.7440825492553286,0.11793250157258299,-0.4579037467999829,-0.5156356900193299,-0.11406998006886662,1.1359638577364604,0.42028911168653604,-0.7087844275233788,26.5
-0.3455732211766638,0.36427234835592676,-1.0391186827555716,-0.2932942300427058,0.18821222130586696,1.7449902971170148,-0.5375681332718376,-0.4503979842370419,-0.5156356900193299,-0.8248813725802624,-2.509995764499681,0.3625389295570029,-1.3345244659586795,43.5
-0.39469195053391243,-0.49559343450001486,-1.0274226581870953,-0.2932942300427058,-0.36853332710832715,0.21384840125893545,0.5666174280971062,-0.5569420477877213,-0.5156356900193299,-0.6353316679105568,-0.8485458100629574,0.4194141089269978,-0.5187037506594405,23.6


### Create Vectors

Use the VectorAssembler to combine all the features (not including the label) into a single column called `features`.

In [0]:
# TODO
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=boston_housing.feature_names, outputCol='features')
vecTrainDF = vecAssembler.transform(trainDF)

Let's now create a UDF to convert our Vector into an Array.

In [0]:
%scala
import org.apache.spark.ml.linalg.Vector
val toArray = udf { v: Vector => v.toArray }
spark.udf.register("toArray", toArray)

import org.apache.spark.ml.linalg.Vector
toArray: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5958/586630716@68dc2c42,ArrayType(DoubleType,false),List(Some(class[value[0]: vector])),None,true,true)
res0: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5958/586630716@68dc2c42,ArrayType(DoubleType,false),List(Some(class[value[0]: vector])),None,true,true)

Save the DataFrame out as a parquet file to DBFS. 

Let's remember to remove the committed and started metadata files in the Parquet folder! Horovod with Petastorm will not work otherwise.

In [0]:
file_path = f"{ml_working_path}/petastorm.parquet"
vecTrainDF.selectExpr("toArray(features) AS features", "label").repartition(8).write.mode("overwrite").parquet(file_path)
[dbutils.fs.rm(i.path) for i in dbutils.fs.ls(file_path) if ("_committed_" in i.name) | ("_started_" in i.name)]

Out[12]: [True, True]

Let's now define our `run_training_horovod` to format our data using Petastorm and distribute the training of our Keras model using Horovod.

In [0]:
# TODO
from petastorm import make_batch_reader
from petastorm.tf_utils import make_petastorm_dataset
import horovod.tensorflow.keras as hvd

abs_file_path = file_path.replace("dbfs:/", "/dbfs/")

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  with make_batch_reader("file://" + abs_file_path, 
                         num_epochs=100, 
                         cur_shard=hvd.rank(), 
                         shard_count= hvd.size()) as reader:
    
    dataset = make_petastorm_dataset(reader).map(lambda x: (tf.reshape(x.features, [-1,13]), tf.reshape(x.label, [-1,1])))
    model = build_model()
    from tensorflow.keras import optimizers
    optimizer = optimizers.Adam(lr=0.001*hvd.size())
    optimizer = hvd.DistributedOptimizer(optimizer)
    
    model.compile(optimizer=optimizer, loss='mse')
    
    checkpoint_dir = f"{ml_working_path}/petastorm_checkpoint_weights_lab.ckpt"
    
    callbacks = [
      hvd.callbacks.BroadcastGlobalVariablesCallback(0),
      hvd.callbacks.MetricAverageCallback(),
      hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
      ReduceLROnPlateau(monitor="loss", patience=10, verbose=1)
    ]

    # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
    if hvd.rank() == 0:
      callbacks.append(ModelCheckpoint(checkpoint_dir, save_weights_only=True))

    history = model.fit(dataset, steps_per_epoch=10, epochs=10, callbacks=callbacks) # (use steps_per_epoch=10)

Finally, let's run our newly define Horovod training function with Petastorm to run across all workers.

In [0]:
# TODO
from sparkdl import HorovodRunner

hr = HorovodRunner(np=-1) # spawn 1 subprocesses on the driver node
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'make_petastorm_dataset', 'ModelCheckpoint', 'hvd', 'make_batch_reader', 'build_model', 'abs_file_path', 'ReduceLROnPlateau', 'tf', 'ml_working_path'}.
The pickled object size is 3038 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_0cd03439190a/id_rsa', 'bash', '/tmp/HorovodRunner_0cd03439190a/launch.sh'].

[1,0]<stderr>:Failed loading Unischema from metadata in %s. Assuming the dataset was not created with Petastorm. Will try to construct from native Parquet schema.
[1,0]<stderr>:Recovering rowgroup information for the entire dataset. This can take a long time for datasets with large number of files. If this dataset was generated by Petastorm (i.e. by using "with materialize_dataset(...)") and you still see this message, this indicates that the materialization did not finish successfully.
[1,0]<stderr>:2022-03-09 11:01:46.277790: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 FMA
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-03-09 11:01:46.284551: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2397220000 Hz
[1,0]<stderr>:2022-03-09 11:01:46.284992: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55be696a0480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2022-03-09 11:01:46.285036: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stderr>:WARNING:tensorflow:AutoGraph could not transform <function run_training_horovod.<locals>.<lambda> at 0x7fcb100c0cb0> and will run it as-is.
[1,0]<stderr>:Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
[1,0]<stderr>:Cause: Unable to locate the source code of <function run_training_horovod.<locals>.<lambda> at 0x7fcb100c0cb0>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
[1,0]<stderr>:To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[1,0]<stderr>:AutoGraph could not transform <function run_training_horovod.<locals>.<lambda> at 0x7fcb100c0cb0> and will run it as-is.
[1,0]<stderr>:Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
[1,0]<stderr>:Cause: Un